# 1. Pointcloud Visualization

##### Idea

To visualize the given LIDAR pointcloud data we first needed to convert the latitude and longitude information to a cartesian coordinate system that has X, Y and Z values to specify a single point. We can do this by using the latitude and longitude information to calculate the UTM coordinates for each point. As a result we will get a value for the easting (X) and northing (Y). The altitude value, which is already given as meters can thus be seen as the Z value in our new coordinate system. After saving the data to a new file we can then use http://lidarview.com/ to visualize the pointcloud to get a first impression of the data.

##### Problems

Our initial thought was that the intesity of each point in our point cloud is very important to determine the the lane markings in the point cloud. In fact, the lane marking are on a flat surface and the marking itself can also be seen as a flat line on the flat surface. Therfore, we cannot use the shape or variance in the point cloud to detect the lane markings. However, the intensity of each LIDAR point should contain enough information to recover the lane marking from the flat surface since the lane marking should have a higher intensity compared to the plain asphalt or concrete of the street. The problem during the visualization was that the visualization tool that we were using could not visualize the intensity of each point. As you can see in the following image, the visualization uses different colors to denote the elevation instead of the intensity of each point. Although the tool had an option for choosing the intensity the visualization did not work, maybe because the intensity was in a wrong format. Finally, we tried to use this tool https://asmaloney.com/software/ which was able to visualize the intensity in a proper manner. You can now easily recognize the lane markings which shows, that we can use the intensity to identify the lane markings.

##### Results
![Full LIDAR Pointcloud Visualization](FullView.png "Full LIDAR Pointcloud View")

![Intensity LIDAR Pointcloud Visualization](IntensityView.png "Intensity LIDAR Pointcloud View")

###### Code

In [1]:
import numpy as np
import pandas as pd

from shapely import wkt

import utm
import geopandas

In [2]:
def read_fuse_to_df(df_name):
    data = []
    
    with open(df_name) as f:  
        line = f.readline()
        while line:
            d = line.split()
            data.append(d)
            line = f.readline()

    a = np.array(data)
    
    pointcloud_df = pd.DataFrame()
    pointcloud_df["Latitude"] = a[:,0]
    pointcloud_df["Longitude"] = a[:,1]
    pointcloud_df["Altitude"] = a[:,2]
    pointcloud_df["Intensity"] = a[:,3]

    return pointcloud_df

In [3]:
def convert_fuse(pointcloud_df, min_x = 0.0, min_y = 0.0, min_z = 0.0):
    pointcloud_df["Latitude"] = pd.to_numeric(pointcloud_df["Latitude"])
    pointcloud_df["Longitude"] = pd.to_numeric(pointcloud_df["Longitude"])
    pointcloud_df["Altitude"] = pd.to_numeric(pointcloud_df["Altitude"])
    pointcloud_df["Intensity"] = pd.to_numeric(pointcloud_df["Intensity"])
    pointcloud_df["Easting"] = pointcloud_df.apply(lambda x: utm.from_latlon(x["Latitude"], x["Longitude"])[0], axis = 1)
    pointcloud_df["Northing"] = pointcloud_df.apply(lambda x: utm.from_latlon(x["Latitude"], x["Longitude"])[1], axis = 1)
    
    if min_x == 0:
        min_x = pointcloud_df["Easting"].min()
    if min_y == 0:
        min_y = pointcloud_df["Northing"].min()   
    if min_z == 0:
        min_z = pointcloud_df["Altitude"].min()    
        
    pointcloud_df["Easting"] = pointcloud_df["Easting"] - min_x    
    pointcloud_df["Northing"] = pointcloud_df["Northing"] - min_y
    pointcloud_df["Altitude"] = pointcloud_df["Altitude"] - min_z
    
    return pointcloud_df, (min_x, min_y, min_z)

In [4]:
pointcloud_df = read_fuse_to_df('./data/pointcloud.fuse')
pointcloud_df, (min_x, min_y, min_z) = convert_fuse(pointcloud_df)

pointcloud_df.head()

,Latitude,Longitude,Altitude,Intensity,Easting,Northing
0,45.903883,11.028414,10.6604,10,108.226321,65.043547
1,45.903683,11.028221,12.6662,5,93.822923,42.445590
2,45.903683,11.028222,12.6415,7,93.938542,42.355143
3,45.903682,11.028224,12.6263,7,94.080686,42.289830
4,45.903681,11.028226,12.6188,7,94.233803,42.220349


In [5]:
xyzi_df = pointcloud_df[["Easting", "Northing", "Altitude", "Intensity"]]
xyzi_df.to_csv("./data/pointcloud.xyz", sep=" ", header=False, index=False)
xyzi_df.head()

,Easting,Northing,Altitude,Intensity
0,108.226321,65.043547,10.6604,10
1,93.822923,42.445590,12.6662,5
2,93.938542,42.355143,12.6415,7
3,94.080686,42.289830,12.6263,7
4,94.233803,42.220349,12.6188,7


# 2. Lane Marking Detection

##### Idea
To identify the lanes in a much better way we first tried to remove all the noise from the data. In our case that means that we wanted to remove points with an intensity that is too high or too low to be a candidate lane marking point. Since different light conditions throughout the day might influence the intensity value, we first calculated the mean and standard deviation intensity for all points. The assumption here is that the lane marking points have a higher intensity value compared to the mean of all pointcloud points. We could have also set an adaptive threshold per pointcloud slice. However, since there was no slice information left we had to use this approach. The nefit of an adaptive threshold would have been that this approach is more robust against different light conditions within the current scene (e.g. a car that is just about to enter a tunnel but has not entered the tunnel yet). The first figure in the results section below shows the lane marking cloud point after the first filtering process.

As you can also see in the figure, there are a lot of points off the road that obviosly do not belong to the lanes. To reduce the number of points that are off the road we used the actual trajectory information in order to remove points that are too far away from the actual trajectory line. We use a threshold of 20 meters to remove all points that are further away. That way we can reduce the number of points from 12306 to 10208. The second figure in the results below shows the remaining points after filtering based on the trajectory lines.

##### Problems
- We do not have a lot of data so we cannot use a deep learning algorithm to identify lanes

##### Results
![First LIDAR Pointcloud Filter](FirstFilter.png "First LIDAR Pointcloud Filter")
![Trajectory LIDAR Pointcloud Filter](TrajectoryFilter.png "Trajectory LIDAR Pointcloud Filter")

##### Code

In [6]:
def convert_to_shape(row):
    return wkt.loads("POINT (" + str(row["Easting"]) + " " + str(row["Northing"]) +" " + str(row["Altitude"]) + ")")

In [7]:
def filter_by_mean_value(pointcloud_df):

    mean = pointcloud_df["Intensity"].mean()
    std = pointcloud_df["Intensity"].std()

    lanes_df = pointcloud_df[pointcloud_df["Intensity"] > mean + 2 * std]
    lanes_df = lanes_df[lanes_df["Intensity"] < mean + 7 * std ]

    print("MEAN FILTER:")
    print("============")
    print("Intensity - Mean value:      ", mean)
    print("Intensity - Std value:       ", std)
    print("Intensity - Lower bound:     ", mean + 2 * std)
    print("Intensity - Upper bound:     ", mean + 7 * std)
    print("Intensity - Filtered points: ", len(lanes_df))
    print("Intensity - Original points: ", len(pointcloud_df))
    print("Intensity - Reduction to %:  ", len(lanes_df)/len(pointcloud_df))
    
    return lanes_df

In [8]:
def get_trajectory_line(trajectory_file='./data/trajectory.fuse', min_x=0.0, min_y=0.0, min_z=0.0):
    trajectory_df = read_fuse_to_df(trajectory_file)
    trajectory_df, (min_x, min_y, min_z) = convert_fuse(trajectory_df, min_x, min_y, min_z)
    trajectory_df[["Easting", "Northing", "Altitude", "Intensity"]].to_csv("./data/trajectory.xyz", index=False)

    line = "LINESTRING("
    for index, row in trajectory_df.iterrows():
        line = line + str(row["Easting"]) + " " + str(row["Northing"]) + " " + str(row["Altitude"]) + ", "
    line = line[:-2] + ")"
    trajectory_line = wkt.loads(line)
    
    return trajectory_line, (min_x, min_y, min_z)

In [9]:
def filter_by_trajectory_line(pointcloud_df, trajectory_line):
    rows = []

    for index, row in pointcloud_df.iterrows():
        distance = trajectory_line.distance(row["Shape"])
        if distance <= 20:
            rows.append(row)

    filtered_lanes_df = pd.DataFrame(rows, columns=['Easting', 'Northing', 'Altitude', 'Intensity', 'Shape'])
    
    print("\nTRAJECTORY FILTER:")
    print("============")
    print("Intensity - Filtered points: ", len(filtered_lanes_df))
    print("Intensity - Original points: ", len(pointcloud_df))
    print("Intensity - Reduction to %:  ", len(filtered_lanes_df)/len(pointcloud_df))
    return filtered_lanes_df

In [10]:
trajectory_line, (min_x, min_y, min_z) = get_trajectory_line('./data/trajectory.fuse',min_x, min_y, min_z)

lanes_df = xyzi_df.copy()
lanes_df = filter_by_mean_value(lanes_df)
lanes_df[['Easting', 'Northing', 'Altitude', 'Intensity']].to_csv("./data/filter_mean.xyz", index=False)
lanes_df['Shape'] = lanes_df.apply(convert_to_shape, axis=1)
lanes_df = filter_by_trajectory_line(lanes_df, trajectory_line)
lanes_df[['Easting', 'Northing', 'Altitude', 'Intensity']].to_csv("./data/filter_trajectory.xyz", index=False)

MEAN FILTER:
Intensity - Mean value:       8.246153096095984
Intensity - Std value:        10.862899760743089
Intensity - Lower bound:      29.97195261758216
Intensity - Upper bound:      84.2864514212976
Intensity - Filtered points:  19048
Intensity - Original points:  430736
Intensity - Reduction to %:   0.04422198283867613

TRAJECTORY FILTER:
Intensity - Filtered points:  13960
Intensity - Original points:  19048
Intensity - Reduction to %:   0.7328853422931542
